In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import Model

In [2]:
m = Model("Hola mundo")

In [9]:
from gamspy import Container, Sum, Set, Parameter, Variable, Alias, Equation, Domain

In [10]:
m = Container()

# Sets

n = Set(m, "n", description="Nodes in the network")
nn = Alias(m, name="nn", alias_with=n)
i = Set(m, "i", domain=n, description="Source nodes / multi-phase")
j = Set(m, "j", domain=n, description="Junction nodes / multi-phase")
pf = Set(m, "pf", domain=n, description="Processing facilities")

arcs = Set(m, "arcs", domain=[n,nn], description="Allowed connections") 

t = Set(m, "t", description="Time periods")
c = Set(m, "c", description="Flow components")
d = Set(m, "d", description="Pipeline diameter options")
s = Set(m, "s", description="Facility sizes")


# Parameters

q_prod = Parameter(m, "Qprod", domain=[i,t,c], description="Production at source node 'i' of component 'c' during time period 't' [mscf per day]")
st_time = Parameter(m, "st_time", domain=i, description="Production start time of source node 'i' ")
capacity = Parameter(m, "capacity", domain=[s, c], description="Capacity for facility size 's' and component 'c' [mscf per day]")
facility_cost = Parameter(m, "facility_cost", domain=s, description="Cost for facility size 's' [$]")
# install_facility_cost = Parameter(m, "install_facility_cost", domain=pf, description="Cost for installing a facility in node 'pf' [$]")
cpipe = Parameter(m, "cpipe", description="Pipeline cost per inch per km [$/(in*km)]")
cpipe_km = Parameter(m, "cpipe_km", domain=d, description="Pipeline cost per km for diameter size 'd' [$/km]")
mult_pipe = Parameter(m, "mult_pipe", domain=d, description="Multiplier for pipeline of diameter 'd'")
diam = Parameter(m, "diam", domain=d, description="Diameter of pipeline option 'd' [inches]")
ir = Parameter(m, "ir", description="Interest rate [%]")

loc_x = Parameter(m, "loc_x", domain=n, description="Position of node 'n' in x-axis [km]")
loc_y = Parameter(m, "loc_y", domain=n, description="Position of node 'n' in y-axis [km]")
dist = Parameter(m, "dist", domain=[n,nn], description="Distance between nodes'n'to 'nn' [km]")

pwell = Parameter(m, "pwell", domain=[i,t], description="Wellhead pressure per source node 'i' at time period 't' [MPa]")
pmin_pf = Parameter(m, "pmin_pf", description="Minimum inlet pressure at processing facility [MPa]")

In [17]:
dist2 = Parameter(m, "dis2t", domain=Domain(n,nn).where[arcs[n,nn]], description="Distance between nodes'n'to 'nn' [km]")

TypeError: All 'domain' elements must be type Set, Alias, UniverseAlias, or str

In [8]:
y_pf = Variable(m, "y_pf", domain=[pf, s, t], type="binary", 
                        description="Equals 1 if a processing facility of size 's' is installed at node 'pf' during time period 't'")
x = Variable(m, "x", domain=[n, nn, d, t], type="binary", 
                description="Equals 1 if a pipeline segment of diameter 'd' between nodes 'n' and 'nn' is installed at time period 't'")
q_inter = Variable(m, "Qinter", domain=[n, nn, d, t, c], description="Flow of component 'c' through pipeline segment between nodes 'n' and 'nn' of diameter 'd' during time period 't' [mscf per day]")
q_process = Variable(m, "Qprocess", domain=[pf, t, c], description="Amount of component 'c' processed at facility 'pf' during time period 't' [mscf per day]")

In [7]:
a = Equation(m, "mass_balance", domain=[n,c,t], description="Mass balance per component at each node")

In [9]:
a[i, c, t] =  q_prod[i, t, c] + Sum([nn, d], q_inter[nn, i, d, t, c]) == Sum([nn, d], q_inter[i, nn, d, t, c])

In [18]:
Sum(Domain(nn, d).where[[arcs[nn, i]]], q_inter[nn, i, d, t, c]).gamsRepr()

"sum((nn,d) $ [ImplicitSet(parent=Set(name='arcs', domain=[Set(name='n', domain=['*']), Alias(name='nn', alias_with=Set(name='n', domain=['*']))]), name='arcs', domain=[Alias(name='nn', alias_with=Set(name='n', domain=['*'])), Set(name='i', domain=[Set(name='n', domain=['*'])])], extension=None, parent_scalar_domains=[])],Qinter(nn,i,d,t,c))"

In [12]:
help(Sum)

Help on class Sum in module gamspy._algebra.operation:

class Sum(Operation)
 |  Sum(
 |      domain: 'Set | Alias | ImplicitSet | Sequence[Set | Alias] | Domain | Condition',
 |      expression: 'Operation | Expression | MathOp | ImplicitSet | ImplicitParameter | ImplicitVariable | int | bool'
 |  )
 |
 |  Represents a sum operation over a domain.
 |
 |  Parameters
 |  ----------
 |  domain : Set | Alias | ImplicitSet | Sequence[Set | Alias], Domain, Condition
 |  expression : (
 |          Expression
 |          | MathOp
 |          | ImplicitVariable
 |          | ImplicitParameter
 |          | int
 |          | bool
 |          | Variable
 |          | Parameter
 |          | Operation
 |      )
 |
 |  Examples
 |  --------
 |  >>> import gamspy as gp
 |  >>> m = gp.Container()
 |  >>> i = gp.Set(m, "i", records=['i1','i2', 'i3'])
 |  >>> v = gp.Variable(m, "v")
 |  >>> e = gp.Equation(m, "e", type="eq")
 |  >>> d = gp.Parameter(m, "d", domain=[i], records=[("i1", 1), ("i2", 2), (

In [11]:
help(type(a))

Help on class Equation in module gamspy._symbols.equation:

class Equation(gams.transfer.syms.container_syms._equation.Equation, gamspy._symbols.symbol.Symbol)
 |  Equation(
 |      container: 'Container | None' = None,
 |      name: 'str | None' = None,
 |      type: 'str | EquationType' = 'regular',
 |      domain: 'Sequence[Set | Alias | str] | Set | Alias | str | None' = None,
 |      definition: 'Variable | Operation | Expression | None' = None,
 |      records: 'Any | None' = None,
 |      domain_forwarding: 'bool' = False,
 |      description: 'str' = '',
 |      uels_on_axes: 'bool' = False,
 |      is_miro_output: 'bool' = False,
 |      definition_domain: 'list | None' = None
 |  )
 |
 |  Represents an Equation symbol in GAMS.
 |  https://gamspy.readthedocs.io/en/latest/user/basics/equation.html
 |
 |  Parameters
 |  ----------
 |  container : Container
 |      Container of the variable.
 |  name : str, optional
 |      Name of the equation. Name is autogenerated by default.
